# Loss Ratio Prediction

In [ ]:
import os
import math
import csv
import collections
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

## Load Dataset

In [ ]:
path_training_dataset = "/kaggle/input/trainingdataset/training_data.csv"
df_data = pd.read_csv(path_training_dataset)

/tmp/ipykernel_47/1709787574.py:2: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(path_training_dataset)


In [ ]:
df_data.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


In [ ]:
df_data.columns

Index(['PolicyNo', 'Policy_Company', 'Policy_Installment_Term',
       'Policy_Billing_Code', 'Policy_Method_Of_Payment',
       'Policy_Reinstatement_Fee_Indicator',
       'Policy_Zip_Code_Garaging_Location', 'Vehicle_Territory',
       'Vehicle_Make_Year', 'Vehicle_Make_Description', 'Vehicle_Performance',
       'Vehicle_New_Cost_Amount', 'Vehicle_Symbol',
       'Vehicle_Number_Of_Drivers_Assigned', 'Vehicle_Usage',
       'Vehicle_Miles_To_Work', 'Vehicle_Days_Per_Week_Driven',
       'Vehicle_Annual_Miles', 'Vehicle_Anti_Theft_Device',
       'Vehicle_Passive_Restraint', 'Vehicle_Age_In_Years',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Bodily_Injury_Limit',
       'Vehicle_Physical_Damage_Limit',
       'Vehicle_Comprehensive_Coverage_Indicator',
       'Vehicle_Comprehensive_Coverage_Limit',
       'Vehicle_Collision_Coverage_Indicator',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Total',
       'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single',
     

### Common Functions

In [ ]:
def get_selected_features(feature_list_file_path):
    with open(feature_list_file_path) as f:
        features_list = f.read().splitlines()
        return features_list

In [ ]:
def strip_white_spaces(input_df):
    columns = list(input_df.columns)
    for column in columns:
        input_df[column] = input_df[column].str.strip()
    return input_df

In [ ]:
def convert_df_objects_to_str(df_dataset, df_columns):
    for column in df_columns:
        if df_dataset[column].dtype == 'object':
            df_dataset[column] = df_dataset[column].astype(str)
    return df_dataset

In [ ]:
def get_num_cat_features(df_dataset, df_columns):
    categorical_variables = []
    numerical_variables = []
    for column in df_columns:
        print(column)
        if df_dataset[column].dtype == 'object':
            categorical_variables.append(column)
        else:
            numerical_variables.append(column)
    return numerical_variables, categorical_variables

In [ ]:
selected_features_train = [
    'Vehicle_Youthful_Good_Student_Code',
    'Vehicle_Driver_Points',
    'Vehicle_Age_In_Years',
    'Driver_Total_Female',
    'Driver_Total_Male',
    'Driver_Total_Teenager_Age_15_19',
    'Driver_Total_College_Ages_20_23',
    'Driver_Total_Young_Adult_Ages_24_29',
    'Driver_Total_Low_Middle_Adult_Ages_30_39',
    'Driver_Total_Middle_Adult_Ages_40_49',
    'Driver_Total_Adult_Ages_50_64',
    'Driver_Total_Senior_Ages_65_69',
    'Driver_Total_Upper_Senior_Ages_70_plus',
    'Driver_Total_Married',
    'Driver_Total_Single',
    'Vehicle_Usage',
    'Vehicle_Miles_To_Work',
    'Vehicle_Territory',
    'Annual_Premium',
    'Loss_Amount']

In [ ]:
df_data = df_data[selected_features_train]

In [ ]:
num_features, cat_features = get_num_cat_features(df_data, list(df_data.columns))

Vehicle_Youthful_Good_Student_Code
Vehicle_Driver_Points
Vehicle_Age_In_Years
Driver_Total_Female
Driver_Total_Male
Driver_Total_Teenager_Age_15_19
Driver_Total_College_Ages_20_23
Driver_Total_Young_Adult_Ages_24_29
Driver_Total_Low_Middle_Adult_Ages_30_39
Driver_Total_Middle_Adult_Ages_40_49
Driver_Total_Adult_Ages_50_64
Driver_Total_Senior_Ages_65_69
Driver_Total_Upper_Senior_Ages_70_plus
Driver_Total_Married
Driver_Total_Single
Vehicle_Usage
Vehicle_Miles_To_Work
Vehicle_Territory
Annual_Premium
Loss_Amount


In [ ]:
num_features

['Vehicle_Driver_Points',
 'Vehicle_Age_In_Years',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount']

In [ ]:
cat_features

['Vehicle_Youthful_Good_Student_Code', 'Vehicle_Usage']

### Cleaning Training Dataset

In [ ]:
df_num = df_data[num_features]
df_num.head()

,Vehicle_Driver_Points,Vehicle_Age_In_Years,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount
0,0,3,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0
1,0,9,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0
2,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0
3,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0
4,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0


In [ ]:
df_cat = df_data[cat_features]
df_cat.head()

,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,Not Eligible for Good Student Credit,Farm
1,Not Eligible for Good Student Credit,Farm
2,Not Eligible for Good Student Credit,Pleasure
3,Not Eligible for Good Student Credit,Pleasure
4,Not Eligible for Good Student Credit,Pleasure


In [ ]:
df_cat_strip = strip_white_spaces(df_cat)

/tmp/ipykernel_47/2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()


In [ ]:
df_data_clean = df_num.join(df_cat, how='outer')

In [ ]:
df_data_clean.head()

,Vehicle_Driver_Points,Vehicle_Age_In_Years,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,0,3,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0,Not Eligible for Good Student Credit,Farm
1,0,9,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0,Not Eligible for Good Student Credit,Farm
2,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0,Not Eligible for Good Student Credit,Pleasure
3,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure
4,0,2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure


### Find useful features

In [ ]:
def get_unique_values_with_count(df_input):
    columns = df_input.columns
    for column in columns:
        unique_value_count = df_input[column].value_counts()
        print(column)
        print()
        print(unique_value_count)
        print('--'*50)

In [ ]:
get_unique_values_with_count(df_data)

Vehicle_Youthful_Good_Student_Code

Vehicle_Youthful_Good_Student_Code
Not Eligible for Good Student Credit    418562
Eligible for Good Student Credit          5869
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
Vehicle_Driver_Points

Vehicle_Driver_Points
0    381289
1     36431
2      5905
3       670
4       108
5        21
6         5
7         2
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
Vehicle_Age_In_Years

Vehicle_Age_In_Years
9    225601
8     28145
7     28094
5     27122
6     26797
4     25740
3     24604
2     23057
1     15271
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Female

Driver_Total_Female
1    238676
0    181914
2      3837
3         4
Name: count, dtype: int64
-----------------------------------------------

### Features to be used in portfolio

- Vehicle_Make_Year_Mean
- Vehicle_Performance_Standard_Minus_all
- Vehicle_Number_Of_Drivers_Assigned_Mean_Ignore_gt_10
- Vehicle_Usage_Pleasure_Minus_all
- Vehicle_Anti_Theft_Device_anything_but_not_applicable
- Vehicle_Age_In_Years_Mean
- Vehicle_Collision_Coverage_Indicator_Y_minus_N
- Driver_Total_Single_Sum_not_include_0
- Driver_Total_Mean
- Driver_Total_Male_Mean
- Driver_Total_Female_Mean
- Driver_Total_Married_Mean
- Driver_Total_Teenager_Age_15_19_Mean
- Driver_Total_College_Ages_20_23_Sum
- Driver_Total_Young_Adult_Ages_24_29_Mean
- Driver_Total_Upper_Senior_Ages_70_plus_Mean
- Annual_Premium_Sum

In [ ]:
selected_features_test = ['Vehicle_Make_Year',
                        'Vehicle_Number_Of_Drivers_Assigned',
                        'Vehicle_Age_In_Years',
                        'Driver_Total_Single',
                        'Driver_Total',
                        'Driver_Total_Male',
                        'Driver_Total_Female',
                        'Driver_Total_Married',
                        'Driver_Total_Teenager_Age_15_19',
                        'Driver_Total_College_Ages_20_23',
                        'Driver_Total_Young_Adult_Ages_24_29',
                        'Driver_Total_Upper_Senior_Ages_70_plus',
                        'Vehicle_Usage',
                        'Vehicle_Anti_Theft_Device',
                        'Vehicle_Performance',
                        'Annual_Premium']

### Create Portfolios in training data like testing portfolios

In [ ]:
df_data.shape

(424431, 20)

In [ ]:
policies_in_portfolios = [1000, 3000, 5000]
percentage_losses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

In [ ]:
df_list_portfolios = []

In [ ]:
df_data_oh = pd.get_dummies(df_data)

In [ ]:
df_data_oh.shape

(424431, 24)

In [ ]:
df_data_oh_columns = list(df_data_oh.columns)

In [ ]:
for i in range(10):
    df_data = df_data_oh.sample(frac=1)
    df_data_loss_zero = df_data[df_data['Loss_Amount'] == 0]
    df_data_loss_non_zero = df_data[df_data['Loss_Amount'] != 0]
    current_df_without_loss = 0
    current_df_with_loss = 0
    for i in range(len(policies_in_portfolios)):
        for j in range(len(percentage_losses)):
            without_loss_size = int(((100 - percentage_losses[j])/100) * policies_in_portfolios[i])
            with_loss_size = int(policies_in_portfolios[i] - without_loss_size )

            df_without_loss = df_data_loss_zero.iloc[current_df_without_loss: current_df_without_loss + without_loss_size]
            df_with_loss = df_data_loss_non_zero.iloc[current_df_with_loss: current_df_with_loss + with_loss_size]

            current_df_without_loss = current_df_without_loss + without_loss_size
            current_df_with_loss = current_df_with_loss + with_loss_size

            df_merge = pd.concat([df_without_loss, df_with_loss])
            df_list_portfolios.append(df_merge)

In [ ]:
len(df_list_portfolios)

330

In [ ]:
df_list_portfolios[0].shape

(1000, 24)

In [ ]:
df_data_final = None

In [ ]:
df_data_oh_columns

['Vehicle_Driver_Points',
 'Vehicle_Age_In_Years',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [ ]:
selected_features_mean = num_features.copy()
selected_features_mean.remove('Annual_Premium')
selected_features_mean.remove('Loss_Amount')
mean_columns = selected_features_mean

sum_columns = ['Annual_Premium']

for cat_f in cat_features:
    for col in list(df_list_portfolios[0].columns):
        if col.startswith(cat_f):
            sum_columns.append(col)

target_column = ['Loss_Amount']

In [ ]:
sum_columns

['Annual_Premium',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [ ]:
ls_final_training = []

In [ ]:
def portfolio_to_features(df_portfolio, is_training=True):
    df_portfolio_mean = df_portfolio[mean_columns]
    df_portfolio_mean = df_portfolio_mean.mean()
    ls_portfolio_mean = df_portfolio_mean.to_list()

    df_portfolio_sum = df_portfolio[sum_columns]
    df_portfolio_sum = df_portfolio_sum.sum()
    ls_portfolio_sum = df_portfolio_sum.to_list()

    if is_training:
        df_portfolio_target = df_portfolio[target_column]
        df_portfolio_target_sum = df_portfolio_target.sum()
        ls_portfolio_target_sum = df_portfolio_target_sum.to_list()

    list_merge = ls_portfolio_mean + ls_portfolio_sum

    if is_training:
        list_merge = list_merge + ls_portfolio_target_sum

    return list_merge

In [ ]:
def features_to_training_df(df_portfolios_list):
    features_list = mean_columns + sum_columns + target_column
    final_list = []
    for single_portfolio in df_portfolios_list:
        ls_portfolio = portfolio_to_features(single_portfolio)
        final_list.append(ls_portfolio)
    df_final_training = pd.DataFrame(final_list, columns = features_list)
    return df_final_training

In [ ]:
df_final_train = features_to_training_df(df_list_portfolios)

In [ ]:
df_final_train.head()

,Vehicle_Driver_Points,Vehicle_Age_In_Years,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,...,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit,Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit,Vehicle_Usage_Business,Vehicle_Usage_Farm,Vehicle_Usage_Pleasure,Vehicle_Usage_Work,Loss_Amount
0,0.107,7.063,0.562,0.566,0.046,0.041,0.073,0.205,0.267,0.338,...,2.890,30.956,325272.37,18.0,982.0,7.0,106.0,528.0,359.0,77269.84
1,0.115,7.148,0.607,0.512,0.045,0.051,0.082,0.218,0.263,0.308,...,3.553,30.791,323636.67,14.0,986.0,5.0,99.0,516.0,380.0,559077.12
2,0.133,7.011,0.599,0.528,0.046,0.039,0.083,0.220,0.262,0.319,...,3.347,31.041,330491.30,13.0,987.0,7.0,103.0,526.0,364.0,107347.74
3,0.124,7.049,0.566,0.544,0.046,0.045,0.086,0.240,0.258,0.297,...,3.399,31.307,328915.17,12.0,988.0,10.0,90.0,529.0,371.0,148324.24
4,0.104,6.992,0.621,0.498,0.040,0.047,0.091,0.215,0.268,0.300,...,3.597,30.995,342443.53,13.0,987.0,8.0,95.0,514.0,383.0,372702.24


### Training the model with Decision Tree

In [ ]:
X_train = df_final_train.loc[:, df_final_train.columns != 'Loss_Amount']
y_train = df_final_train['Loss_Amount']

In [ ]:
X_train.shape, y_train.shape

((330, 23), (330,))

In [ ]:
X_train.columns

Index(['Vehicle_Driver_Points', 'Vehicle_Age_In_Years', 'Driver_Total_Female',
       'Driver_Total_Male', 'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Driver_Total_Married',
       'Driver_Total_Single', 'Vehicle_Miles_To_Work', 'Vehicle_Territory',
       'Annual_Premium',
       'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
       'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
       'Vehicle_Usage_Business', 'Vehicle_Usage_Farm    ',
       'Vehicle_Usage_Pleasure', 'Vehicle_Usage_Work    '],
      dtype='object')

In [ ]:
decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [ ]:
y_train_pred = decision_tree_regressor.predict(X_train)

In [ ]:
mae = mean_absolute_error(y_train, y_train_pred)
mae

0.0

In [ ]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.005, max_depth = 5, alpha = 5, n_estimators = 200)
xg_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

XGBRegressor(alpha=5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, ...)

In [ ]:
y_train_pred_xg = xg_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [ ]:
mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg

396922.2296212121

## Train Using KNN Model

In [ ]:
knn_regressor = KNeighborsRegressor(5)
knn_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

KNeighborsRegressor()

In [ ]:
y_train_pred_knn = knn_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [ ]:
mae_knn = mean_absolute_error(y_train, y_train_pred_knn)
mae_knn

291352.27920000005

In [ ]:
def load_test_portfolios():
    path_testing_dataset = "/kaggle/input/testingportfolios/testing_portfolios"
    portfolio_files = os.listdir(path_testing_dataset)
    portfolio_dict = {}
    for i in range(len(portfolio_files)):
        portfolio_id = int(portfolio_files[i].split('.')[0][15:])
        portfolio_dict[portfolio_id] = portfolio_files[i]
    od = collections.OrderedDict(sorted(portfolio_dict.items()))
    po_list = []
    for key,val in od.items():
        po_list.append(val)
    df_portfolio_list = []
    for portfolio_file in po_list:
        file_path = os.path.join(path_testing_dataset, portfolio_file)
        df_test_portfolio = pd.read_csv(file_path)
        test_num_features = num_features.copy()
        test_num_features.remove('Loss_Amount')

        df_test_num = df_test_portfolio[test_num_features]
        df_test_cat = df_test_portfolio[cat_features]
        df_test_cat_strip = strip_white_spaces(df_test_cat)
        df_test_portfolio = df_test_num.join(df_test_cat_strip, how='outer')
        df_test_portfolio = pd.get_dummies(df_test_portfolio)
        df_test_portfolio = df_test_portfolio.reindex(sorted(X_train.columns), axis=1)

        df_portfolio_list.append(df_test_portfolio)

    result_dict = collections.OrderedDict()
    for i in range(len(po_list)):
        portfolio_id = po_list[i].split('.')[0][5:]
        result_dict[portfolio_id] = df_portfolio_list[i]
    return result_dict

In [ ]:
def features_to_testing_df(df_portfolios_ordered_dict):
    features_list = mean_columns + sum_columns
    features_list.insert(0, 'ID')
    final_list = []
    for portfolio_id, single_portfolio in df_portfolios_ordered_dict.items():
        ls_portfolio = portfolio_to_features(single_portfolio, False)
        ls_portfolio.insert(0, portfolio_id)
        final_list.append(ls_portfolio)
    df_final_testing = pd.DataFrame(final_list, columns = features_list)
    return df_final_testing

In [ ]:
test_portfolios_dict = load_test_portfolios()

/tmp/ipykernel_47/2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
/tmp/ipykernel_47/2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
/tmp/ipykernel_47/2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
for key, value in test_portfolios_dict.items():
    print(key, value.shape)

portfolio_1 (1000, 23)
portfolio_2 (999, 23)
portfolio_3 (1000, 23)
portfolio_4 (997, 23)
portfolio_5 (998, 23)
portfolio_6 (1000, 23)
portfolio_7 (1000, 23)
portfolio_8 (998, 23)
portfolio_9 (998, 23)
portfolio_10 (998, 23)
portfolio_11 (1000, 23)
portfolio_12 (999, 23)
portfolio_13 (999, 23)
portfolio_14 (997, 23)
portfolio_15 (1000, 23)
portfolio_16 (999, 23)
portfolio_17 (998, 23)
portfolio_18 (999, 23)
portfolio_19 (998, 23)
portfolio_20 (997, 23)
portfolio_21 (1000, 23)
portfolio_22 (999, 23)
portfolio_23 (1000, 23)
portfolio_24 (1000, 23)
portfolio_25 (998, 23)
portfolio_26 (1000, 23)
portfolio_27 (998, 23)
portfolio_28 (999, 23)
portfolio_29 (998, 23)
portfolio_30 (998, 23)
portfolio_41 (1000, 23)
portfolio_42 (1000, 23)
portfolio_43 (998, 23)
portfolio_44 (999, 23)
portfolio_45 (999, 23)
portfolio_46 (999, 23)
portfolio_47 (998, 23)
portfolio_48 (1000, 23)
portfolio_49 (999, 23)
portfolio_50 (999, 23)
portfolio_51 (999, 23)
portfolio_52 (998, 23)
portfolio_53 (996, 23)
portfol

In [ ]:
df_test_final = features_to_testing_df(test_portfolios_dict)

In [ ]:
def generate_test_results(regressor, X_test, output_dir):
    # for each policy
    result_data = collections.OrderedDict()
    portfolio_ids_df = X_test.iloc[:, 0:1]
    X_test_features = X_test.iloc[:, 1:]

    y_pred = regressor.predict(np.nan_to_num(X_test_features.to_numpy()))
    loss_ratio = y_pred/X_test['Annual_Premium']
    loss_ratio_log = np.log(loss_ratio)
    loss_ratio_log_df = loss_ratio_log.to_frame(name='ln_LR')
    result_df = portfolio_ids_df.join(loss_ratio_log_df, how='outer')

    result_df.to_csv('output.csv', index=False)

In [ ]:
generate_test_results(knn_regressor, df_test_final, '')